In [134]:
import numpy as np
import pandas as pd
from Evaluator import ModelEvaluator
import torch
import random
from matplotlib import pylab as plt
from IPython.display import clear_output
import copy
import math

In [135]:
class MalwareEnv():
    def __init__(self,epoch,origin):
        # Actions we can take, decrease, increse
        self.action_space = 20  
        # Set start   
        self.state = self.generateWorld()
        #model to evaluate the reward
        self.evaluator=ModelEvaluator()
        #units to move
        self.block=self.calculateBlock(epoch, origin)
    
    
    def generateWorld(self):
        
        valuesToTest=np.array([])
        
        for x in range(10):
            valuesToTest=np.append(valuesToTest, random.randint(-100,100))
        
        return valuesToTest 
    
    def  calculateBlock(self,epoca,origin):
        
        #return 10
        return math.ceil(epoca/origin*60)  
        
    def step(self, action):
        
        if(action<9):
            self.state[action]+=self.block
        elif(action>=9 and action<19):
            self.state[action-9]-=self.block
            
        
    def reward(self):
        reward=-1
        value=self.state.tolist()
        result=self.evaluator.test(value)
        result=np.array2string(result)
        print(result)
        if(result=="['malicious']"):
                 reward=+10
                 
        return reward    

    def render_np(self):
        return self.state
    

In [136]:
l1 = 10
l2 = 24
l3 = 72
l4 = 20
model = torch.nn.Sequential(
    torch.nn.Linear(l1, l2),
    torch.nn.ReLU(),
    torch.nn.Linear(l2, l3),
    torch.nn.ReLU(),
    torch.nn.Linear(l3,l4)
)
#device=torch.device("cuda:0" if torch.cuda.is_available else "cpu")
model.cuda()
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
gamma = 0.9
epsilon = 1.0

In [137]:
initialeEpoch =500
epochs = copy.copy(initialeEpoch)
losses = [] 
resistance=0
effort=0

for i in range(epochs): 
    env = MalwareEnv(epochs,initialeEpoch) 
    state_ = env.render_np().reshape(1,10) + np.random.rand(1,10)/50.0 
    state1 = torch.from_numpy(state_).float() 
    status = 1 
    while(status == 1): 
        qval = model(state1.cuda()) 
        qval_ = qval.data.cpu().numpy()
        if (random.random() < epsilon):
            action_ = np.random.randint(0,20)
        else:
            action_ = np.argmax(qval_)
        
        env.step(action_) 
        state2_ = env.render_np().reshape(1,10) + np.random.rand(1,10)/50.0
        state2 = torch.from_numpy(state2_).float() 
        reward = env.reward()
        with torch.no_grad():
            newQ = model(state2.reshape(1,10).cuda())
        maxQ = torch.max(newQ) 
        if reward == -1: 
            Y = reward + (gamma * maxQ)
        else:
            Y = reward
        Y = torch.Tensor([Y]).detach()
        X = qval.squeeze()[action_] 
        loss = loss_fn(X.cuda(), Y.cuda()) 
        print(resistance, i, loss.item())
        clear_output(wait=True)
        optimizer.zero_grad()
        loss.backward()
        if(effort<1000):  
            losses.append(loss.item())
        optimizer.step()
        state1 = state2
        if reward != -1: 
            status = 0
            resistance=0
            effort=0
        resistance+=1
        effort=+1
        if(resistance>120):
            resistance=0
            env = MalwareEnv(epochs,initialeEpoch)
    if epsilon > 0.1: 
        epsilon -= (1/epochs)

KeyboardInterrupt: 